In [44]:
# step 1: load csv data into pandas
#step 2: use requests library from pandas to call external api
#step 3: store the response into pandas series and then save as csv



In [73]:
path = "C:\\Users\\michelle\\Desktop\\BIG_POS_MKTG_TRIAL_CODE\\python_programs\\airportseu.csv"



In [74]:
#import the following libraries

import pandas as pd
import requests

import json
import time
from pandas.io.json import json_normalize

df = pd.read_csv(path)

df.columns = df.columns.to_series().apply(lambda x: x.strip())


#df = df["name"]- you can only have double codes for one parameter

#df = df[['id', 'ident','type','name', 'latitude_deg', 'longitude_deg', 'continent','iso_country', 'iso_region','municipality',
 #       'iata_code', 'home_link', 'wikipedia_link', 'keywords']] 
#df.head()



In [76]:
#subset cols from dataset
df = df[['iso_country','type', 'name','continent','municipality','iata_code','home_link', 'latitude_deg', 'longitude_deg']]
print(df.head())

  iso_country      type                       name continent  \
0          GB    closed               RAF Calveley        EU   
1          GB    closed           RNAS/RAF Calshot        EU   
2          AD  heliport              Camí Heliport        EU   
3          AD  heliport  Andorra la Vella Heliport        EU   
4          GB    closed             RAF Castletown        EU   

       municipality iata_code home_link  latitude_deg  longitude_deg  
0          Cheshire       NaN       NaN     53.113333      -2.603889  
1         Hampshire       NaN       NaN     50.819913      -1.306772  
2        La Massana       NaN       NaN     42.546257       1.519160  
3  Andorra La Vella       ALV       NaN     42.511174       1.533551  
4         Caithness       NaN       NaN     58.584720      -3.348169  


In [77]:
#use API for reverse geocoding function. api_key will be generated from this external web service

#convert coordinates to a readable address & location


def get_reverse_geocode_data(row):
    try:
        my_api_key = '9be8a2c17570bd'
        
        #row refers to each column data pair of lat & lon
        
        url = 'https://eu1.locationiq.org/v1/reverse.php?key=' + my_api_key + '&lat=' + str(row['latitude_deg']) + '&lon=' + str(row['longitude_deg']) + '&format=json'
        response = (requests.get(url).text)
        response_json = json.loads(response)
        
        #timer is needed to control the calls to api
        time.sleep(0.5)
        return response_json
    
    except Exception as e:
        raise e           


In [78]:
df['API_Response'] = df.apply(get_reverse_geocode_data, axis=1)

C:\Users\michelle\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
#check json response from api...data output means successful!
print(df['API_Response'].head())

0    {'place_id': '84670235', 'licence': '© Locatio...
1    {'place_id': '76136248', 'licence': '© Locatio...
2    {'place_id': '331758300147', 'osm_type': 'way'...
3    {'place_id': '330432581290', 'licence': '© Loc...
4    {'place_id': '122196727', 'licence': '© Locati...
Name: API_Response, dtype: object


In [81]:
#flatten the response into columns and keep those required only. in my case i decided to keep lat, lon & display_name cols

new_df = json_normalize(df['API_Response'])
new_df = new_df[['lat', 'lon', 'display_name']]
print(new_df.head(10))

                lat                lon  \
0        53.1103566         -2.6023281   
1        50.8198482   -1.3074849129902   
2         42.546291           1.519102   
3          42.51124            1.53358   
4  58.5895083580109  -3.34802964801382   
5        36.1287822        -5.44105465   
6        41.3336537   19.7909198950225   
7       42.60520335      20.9100823964   
8       40.60635885   19.4260322927939   
9        40.0857552          20.152102   

                                        display_name  
0  Green Lane, Wardle, Cheshire East, North West ...  
1  Calshot Castle, Jack Maynard Road, Fawley, New...  
2   Helipad La Massana, La Massana, Massana, Andorra  
3  Andorra la Vella Heliport, Andorra la Vella, A...  
4  C1033, Castlehill, Caithness, Scotland, KW14 8...  
5  AEI, N-350, San García, Algeciras, Campo de Gi...  
6  Fushë Sporti, Rruga e Dafinave, Lapraka, Tiran...  
7  Aeroporti i Korroticës së Ulët, Aeroporti, Lag...  
8  Aeroporti i Mifoit, A2, Levan, Bashkia 

In [82]:
#need to convert datatype of column from api response from default object to float type, else merge function wont work

new_df['lat']= new_df['lat'].astype(float)

In [83]:
#df = df[['iso_country', 'latitude_deg', 'longitude_deg']]
df = df[['iso_country','type', 'name','continent','municipality','iata_code','home_link', 'latitude_deg', 'longitude_deg']]
print(df.head(10))

  iso_country      type                            name continent  \
0          GB    closed                    RAF Calveley        EU   
1          GB    closed                RNAS/RAF Calshot        EU   
2          AD  heliport                   Camí Heliport        EU   
3          AD  heliport       Andorra la Vella Heliport        EU   
4          GB    closed                  RAF Castletown        EU   
5          ES  heliport              Algeciras Heliport        EU   
6          AL    closed                Lapraka Heliport        EU   
7          AL    closed  Aeroporti i Korroticës së Ulët        EU   
8          AL    closed              Aeroporti i Mifoit        EU   
9          AL    closed             Gjirokastër Airport        EU   

         municipality iata_code  \
0            Cheshire       NaN   
1           Hampshire       NaN   
2          La Massana       NaN   
3    Andorra La Vella       ALV   
4           Caithness       NaN   
5           Algeciras       AE

In [85]:
#merge 2 data-frames : join results of new dataframe to old one by using foreign key

dfinal = df.merge(new_df , how='inner', left_on='latitude_deg', right_on='lat')

In [86]:
dfinal.head()

,iso_country,type,name,continent,municipality,iata_code,home_link,latitude_deg,longitude_deg,lat,lon,display_name
0,BA,small_airport,Sportski Aerodrom Zalužani,EU,Zalužani,NaN,NaN,44.848200,17.222826,44.848200,17.2228259,"Sportski aerodrom Zalužani, Trkaća staza Zaluž..."
1,BG,small_airport,Trustikovo Airport,EU,Trustikovo,NaN,NaN,42.414635,27.294455,42.414635,27.2944546,"Trustikovo Airport, 79, Sredets, Burgas, 8129,..."
2,BG,small_airport,Kozloduy Airport,EU,Kozloduy,NaN,NaN,43.763134,23.722031,43.763134,23.7220306,"Kozloduy Airport, 11, Козлодуй, Kozlodui, Vrat..."
3,BG,small_airport,Ostrovo,EU,NaN,NaN,NaN,43.693874,26.579964,43.693874,26.5799644,"Ostrovo Airfield, 4902, Zavet, Razgrad, Bulgaria"
4,BG,small_airport,Staro Selishte,EU,Staro Selishte,NaN,NaN,43.650278,26.754167,43.650278,26.7541667,"Staro Selishte, Янтра, Ludogortsi, Isperih, Ra..."


In [6]:
#from pyspark.sql import SQLContext

from pyspark import SparkContext
sc= SparkContext()

In [7]:
#import pandas  as pd

#importing the data and creating the rdd with spark NOT pandas
#import urllib
#f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

path = "C:\\Users\\michelle\\Desktop\\BIG_POS_MKTG_TRIAL_CODE\\python_programs\\airports.csv"
#data = pd.read_csv(path)
raw_data = sc.textFile(path).cache()

In [37]:
#ini a spark dataframe. This is = to table in sql & data frame in pandas

#this is the spark entry point into sql functionality
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [22]:
#convert an RDD of row objects (cols) to data frames. Then infer schema by looking at first row

from pyspark.sql import Row

csv_data = raw_data.map(lambda line: line.split(","))


In [23]:
row_data = csv_data.map(lambda p: Row(
id = p[0],
ident = p[1],
type = p[2],
name = p[3],
    latitude_deg = p[4],
    longitude_deg = p[5],
    elevation_ft = p[6],
    continent = p[7],
    iso_country = p[8],
    iso_region = p[9],
    municipality = p[10],
    scheduled_service = p[11],
    gps_code = p[12],
    iata_code = p[13],
    local_code = p[14],
    home_link = p[15],
    wikipedia_link = p[16],
    keywords = p[17]
))

In [32]:
airports_df = sqlContext.createDataFrame(row_data)

airports_df.registerTempTable("airports")

# airports_df.select('iso_region').distinct().count()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.T

In [19]:
#airports_df = sqlContext.createDataFrame(row_data)

from pyspark.sql.types import *

#sc.CreateDataFrame(row_data, schema).show()

In [20]:
sc.CreateDataFrame(row_data, schema).show()

AttributeError: 'SparkContext' object has no attribute 'CreateDataFrame'